<a href="https://colab.research.google.com/github/dipeshpandya/neural-network-challenge-2/blob/main/Copy_of_attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [21]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [22]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [ ]:
attrition_df.dtypes

In [24]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
print(y_df['Attrition'].value_counts())
print(y_df['Department'].value_counts())

Attrition
No     1233
Yes     237
Name: count, dtype: int64
Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64


In [ ]:
# Create a list of at least 10 column names to use as X data
X_data = ['Education','Age','DistanceFromHome','JobSatisfaction','OverTime','StockOptionLevel','WorkLifeBalance','YearsAtCompany', 'YearsSinceLastPromotion','NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[X_data]

# Show the data types for X_df
X_df.dtypes


In [26]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=10)

In [27]:
X_train['OverTime'].value_counts()

,count
OverTime,
No,791
Yes,311


In [28]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
# transform True to 1 and false to 0
X_train = X_train.astype(int)
X_test = X_test.astype(int)

In [29]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary


In [30]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [ ]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
department_encoder = OneHotEncoder(drop='first', handle_unknown='ignore',sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(np.array(y_train['Department']).reshape(-1, 1))

# Create two new variables by applying the encoder to the training and testing data
dept_y_train_encoded = department_encoder.transform(np.array(y_train['Department']).reshape(-1, 1))
dept_y_test_encoded = department_encoder.transform(np.array(y_test['Department']).reshape(-1, 1))
dept_y_test_encoded

In [ ]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(drop='first', handle_unknown='ignore',sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(np.array(y_train['Attrition']).reshape(-1, 1))

# Create two new variables by applying the encoder
# to the training and testing data
attr_y_train_encoded = attrition_encoder.transform(np.array(y_train['Attrition']).reshape(-1, 1))
attr_y_test_encoded = attrition_encoder.transform(np.array(y_test['Attrition']).reshape(-1, 1))
attr_y_test_encoded

## Create, Compile, and Train the Model

In [35]:
# Find the number of columns in the X training data
cols = X_train.shape[1]

# Create the input layer
input = layers.Input(shape=(cols, ), name='input_layer')


# Create at least two shared layers
shared1 = layers.Dense(64, activation="relu",name='shared_layer_1')(input)
shared2 = layers.Dense(128, activation="relu",name='shared_layer_2')(shared1)

In [36]:
# Create a branch for Department
# with a hidden layer and an output layer

department_input = layers.Dense(64, activation='relu')(shared2)

# Create the hidden layer
hidden_dept= layers.Dense(64, activation='relu')(department_input)
#hidden_dept= layers.Dense(cols, activation='relu')(department_input)

# Create the output layer
#department_output = layers.Dense(1, activation='relu', name = "dept_output")(hidden_dept)
#department_output = layers.Dense(2, activation='softmax', name = "dept_output")(hidden_dept)
department_output = layers.Dense(cols, activation='softmax', name = "dept_output")(hidden_dept)

In [37]:
# Create a branch for Attrition
# with a hidden layer and an output layer
attrition_input = layers.Dense(64, activation='relu')(shared2)


# Create the hidden layer
hidden_attr= layers.Dense(64, activation='relu')(attrition_input)
#hidden_attr= layers.Dense(cols, activation='relu')(attrition_input)

# Create the output layer
#attrition_output = layers.Dense(1, activation='sigmoid', name = "attr_output")(hidden_attr)
attrition_output = layers.Dense(cols, activation='sigmoid', name = "attr_output")(hidden_attr)

In [38]:
# Create the model
model = Model(inputs=input, outputs=[attrition_output, department_output])

# Compile the model
model.compile(optimizer='adam', loss={
        'attr_output': 'binary_crossentropy',
        'dept_output': 'categorical_crossentropy'},
              metrics=[['accuracy'], ['accuracy']])

# Summarize the model
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 11)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer_1 (Dense)    │ (None, 64)             │            768 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shared_layer_2 (Dense)    │ (None, 128)            │          8,320 │ shared_layer_1[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 64)             │          8,256 │ shared_layer_2[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 64)             │          8,256 │ shared_layer_2[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 64)             │          4,160 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 64)             │          4,160 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attr_output (Dense)       │ (None, 11)             │            715 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dept_output (Dense)       │ (None, 11)             │            715 │ dense_5[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 35,350 (138.09 KB)

 Trainable params: 35,350 (138.09 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
# Train the model with the training data
model.fit(X_train_scaled, [attr_y_train_encoded, dept_y_train_encoded], epochs=100, batch_size=32)

Epoch 1/100


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 11)

In [ ]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, [attr_y_test_encoded, dept_y_test_encoded])
results
model_loss1, model_accuracy1, model_accuracy2 = model.evaluate(X_test_scaled,[attr_y_test_encoded, dept_y_test_encoded],verbose=2)

# Display evaluation results
print(f"Loss: {model_loss1}, Attrition Accuracy: {model_accuracy1},Department Accuracy: {model_accuracy2} ")

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric on his data because the classes viz. Attrition and Department are not evenly distributed
2. I used sigmoid activation function for Attrition and softmax for Department as Attrition is binary classification whereas Department is multi-class classification
3.